# Exp T1-spinodal: Measuring λ↑ (Ordered → Collapse Threshold)

## 目的
Ordered状態からλを段階的に上げ、崩壊が起きる閾値（λ↑）を測定。

## 実験設計
- **Phase 1 (Init)**: λ=0.35で50ep → Ordered状態確立
- **Phase 2 (Sweep)**: λを段階的に上昇
  - 0.54 → 0.56 → 0.58 → 0.60 → 0.62 → 0.64 → 0.66 → 0.68 → 0.70
  - 各λで10ep保持
- **崩壊判定**: error ≥ 0.40 が3回連続 → 停止、λ↑を記録
- **η**: 0.4
- **Seeds**: 10（スクリーニング）

## 科学的目標
- λ↑の分布を測定
- T2-spinodal（λ↓）と組み合わせてヒステリシス幅を定量化

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_T_spinodal'
NOTEBOOK_ID = 'T1-spinodal'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Notebook: {NOTEBOOK_ID} (λ↑ measurement, η=0.4)')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# Experiment parameters
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

# Spinodal experiment specific
NOISE_RATE = 0.4
INIT_LAMBDA = 0.35
INIT_EPOCHS = 50
INIT_SUCCESS_THRESHOLD = 0.30

# λ sweep parameters (upward for λ↑)
LAMBDA_SWEEP = [0.54, 0.56, 0.58, 0.60, 0.62, 0.64, 0.66, 0.68, 0.70]
EPOCHS_PER_LAMBDA = 10
COLLAPSE_THRESHOLD = 0.40  # error ≥ this = collapsed
COLLAPSE_PERSIST = 3       # consecutive epochs to confirm

N_SEEDS = 10  # Screening first

print(f'η = {NOISE_RATE}')
print(f'Init: λ={INIT_LAMBDA} for {INIT_EPOCHS}ep')
print(f'Sweep: {LAMBDA_SWEEP}')
print(f'Per λ: {EPOCHS_PER_LAMBDA}ep')
print(f'Collapse: error≥{COLLAPSE_THRESHOLD} for {COLLAPSE_PERSIST} consecutive')
print(f'Seeds: {N_SEEDS}')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, lam, cached_gv_ref):
    crit = nn.CrossEntropyLoss()
    model.train()
    step = cached_gv_ref['step']
    cached_gv = cached_gv_ref['gv']
    
    for x, _, idx in train_loader:
        x, idx = x.to(device), idx.to(device)
        bn, bc = noisy_t[idx], clean_t[idx]
        
        opt.zero_grad()
        loss_s = crit(model(x), bn)
        loss_s.backward(retain_graph=True)
        gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        if step % K == 0 or cached_gv is None:
            opt.zero_grad()
            loss_v = crit(model(x), bc)
            loss_v.backward()
            cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        gs_n = gs / (gs.norm() + 1e-12)
        gv_n = cached_gv / (cached_gv.norm() + 1e-12)
        
        g_mix = (1 - lam) * gs_n + lam * gv_n
        opt.zero_grad()
        i = 0
        for p in model.parameters():
            n = p.numel()
            p.grad = g_mix[i:i+n].view(p.shape).clone()
            i += n
        opt.step()
        step += 1
    
    if sched is not None:
        sched.step()
    
    cached_gv_ref['step'] = step
    cached_gv_ref['gv'] = cached_gv

In [ ]:
def run_spinodal_up(seed, train_loader, test_loader, clean_labels, noisy_labels):
    """Run spinodal experiment: ordered init → sweep λ upward → find collapse point"""
    
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    
    set_seed(seed)
    model = get_resnet18().to(device)
    opt = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    sched = optim.lr_scheduler.MultiStepLR(opt, [50, 75], 0.1)
    
    cached_gv_ref = {'step': 0, 'gv': None}
    trajectory = []
    
    # Phase 1: Initialize to ordered state
    print(f'    Phase 1: Init (λ={INIT_LAMBDA}, {INIT_EPOCHS}ep)...')
    for ep in range(INIT_EPOCHS):
        train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, INIT_LAMBDA, cached_gv_ref)
        if (ep + 1) % 10 == 0:
            err = 1 - evaluate(model, test_loader)
            trajectory.append({'phase': 'init', 'lambda': INIT_LAMBDA, 'epoch': ep+1, 'error': err})
            print(f'      Ep {ep+1}: err={err:.4f}')
    
    init_error = 1 - evaluate(model, test_loader)
    
    if init_error > INIT_SUCCESS_THRESHOLD:
        print(f'    ⚠️ Init failed: {init_error:.4f}')
        return {
            'seed': seed, 'init_error': init_error, 'init_success': False,
            'lambda_up': None, 'collapsed': False, 'trajectory': trajectory
        }
    
    print(f'    ✅ Init OK: {init_error:.4f}')
    print(f'    Phase 2: λ sweep upward...')
    
    # Phase 2: Sweep λ upward
    lambda_up = None
    collapse_count = 0
    total_ep = INIT_EPOCHS
    
    for lam in LAMBDA_SWEEP:
        print(f'      λ={lam:.2f}: ', end='')
        
        for ep in range(EPOCHS_PER_LAMBDA):
            train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, lam, cached_gv_ref)
            total_ep += 1
            err = 1 - evaluate(model, test_loader)
            trajectory.append({'phase': 'sweep', 'lambda': lam, 'epoch': total_ep, 'error': err})
            
            # Check collapse
            if err >= COLLAPSE_THRESHOLD:
                collapse_count += 1
                if collapse_count >= COLLAPSE_PERSIST:
                    lambda_up = lam
                    print(f'💀 COLLAPSED at ep {total_ep} (err={err:.4f})')
                    break
            else:
                collapse_count = 0
        
        if lambda_up is not None:
            break
        
        final_err = 1 - evaluate(model, test_loader)
        print(f'err={final_err:.4f} (stable)')
    
    if lambda_up is None:
        print(f'    ⏱️ No collapse up to λ={LAMBDA_SWEEP[-1]}')
    
    return {
        'seed': seed,
        'init_error': init_error,
        'init_success': True,
        'lambda_up': lambda_up,
        'collapsed': lambda_up is not None,
        'final_lambda': lam,
        'final_error': 1 - evaluate(model, test_loader),
        'trajectory': trajectory
    }

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

m = get_resnet18().to(device)
for _ in range(10): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done_seeds = set()

if os.path.exists(ckpt):
    results = json.load(open(ckpt))
    done_seeds = {r['seed'] for r in results}
    print(f'Loaded: {len(done_seeds)} done')

for seed in range(N_SEEDS):
    if seed in done_seeds:
        continue
    
    run_num = len(results) + 1
    print(f'\n{"="*60}')
    print(f'[{run_num}/{N_SEEDS}] Seed {seed} | {NOTEBOOK_ID}')
    print(f'{"="*60}')
    
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    t0 = time.time()
    result = run_spinodal_up(seed, train_loader, test_loader, clean_labels, noisy_labels)
    dt = time.time() - t0
    
    result['experiment_id'] = f'{NOTEBOOK_ID}-{seed:03d}'
    result['eta'] = NOISE_RATE
    result['time'] = dt
    
    results.append(result)
    done_seeds.add(seed)
    
    # Summary
    if result['collapsed']:
        status = f"💀 λ↑={result['lambda_up']:.2f}"
    else:
        status = f"⏱️ No collapse (up to λ={result['final_lambda']:.2f})"
    print(f'\n  {status} | {dt/60:.1f}min')
    
    # Stats
    collapsed_results = [r for r in results if r.get('collapsed', False)]
    if collapsed_results:
        lambdas = [r['lambda_up'] for r in collapsed_results]
        print(f'  λ↑ stats: n={len(lambdas)}, mean={np.mean(lambdas):.3f}, std={np.std(lambdas):.3f}')
    
    json.dump(results, open(ckpt, 'w'), indent=2)
    
    remaining = N_SEEDS - run_num
    print(f'  ETA: {remaining*dt/3600:.1f}h')
    
    torch.cuda.empty_cache()

print('\n' + '='*60 + f'\n{NOTEBOOK_ID} DONE\n' + '='*60)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

json.dump(results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2)
df = pd.DataFrame([{k: v for k, v in r.items() if k != 'trajectory'} for r in results])
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print('='*60)
print(f'{NOTEBOOK_ID} SUMMARY | η={NOISE_RATE}')
print('='*60)

n_total = len(df)
n_collapsed = df['collapsed'].sum()
print(f'\n📊 Results: {n_collapsed}/{n_total} collapsed ({100*n_collapsed/n_total:.0f}%)')

if n_collapsed > 0:
    lambda_ups = df[df['collapsed'] == True]['lambda_up'].values
    print(f'\n📈 λ↑ Distribution:')
    print(f'   Mean: {lambda_ups.mean():.3f}')
    print(f'   Std:  {lambda_ups.std():.3f}')
    print(f'   Min:  {lambda_ups.min():.2f}')
    print(f'   Max:  {lambda_ups.max():.2f}')
else:
    print('\n⚠️ No collapse events observed!')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. λ↑ histogram
ax = axes[0]
collapsed_df = df[df['collapsed'] == True]
if len(collapsed_df) > 0:
    ax.hist(collapsed_df['lambda_up'], bins=np.arange(0.52, 0.72, 0.02), 
            color='red', alpha=0.7, edgecolor='black')
    ax.axvline(collapsed_df['lambda_up'].mean(), color='darkred', linestyle='--', 
               linewidth=2, label=f"Mean λ↑={collapsed_df['lambda_up'].mean():.3f}")
ax.set_xlabel('λ↑ (Collapse Threshold)', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('λ↑ Distribution (Ordered → Collapse)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Sample trajectories
ax = axes[1]
for r in results[:5]:
    if r['trajectory']:
        sweep_traj = [t for t in r['trajectory'] if t['phase'] == 'sweep']
        if sweep_traj:
            eps = [t['epoch'] for t in sweep_traj]
            errs = [t['error'] for t in sweep_traj]
            color = 'red' if r['collapsed'] else 'blue'
            ax.plot(eps, errs, alpha=0.7, color=color, linewidth=1.5,
                   label=f"seed {r['seed']}: λ↑={r.get('lambda_up', 'N/A')}")

ax.axhline(COLLAPSE_THRESHOLD, color='orange', linestyle='--', linewidth=2, label=f'Collapse ({COLLAPSE_THRESHOLD})')
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title('Error Trajectories During λ Sweep', fontsize=12)
ax.legend(loc='upper left', fontsize=8)
ax.grid(True, alpha=0.3)

# 3. Error vs λ
ax = axes[2]
for r in results[:5]:
    if r['trajectory']:
        sweep_traj = [t for t in r['trajectory'] if t['phase'] == 'sweep']
        if sweep_traj:
            lambdas = [t['lambda'] for t in sweep_traj]
            errs = [t['error'] for t in sweep_traj]
            color = 'red' if r['collapsed'] else 'blue'
            ax.scatter(lambdas, errs, alpha=0.5, color=color, s=20)

ax.axhline(COLLAPSE_THRESHOLD, color='orange', linestyle='--', linewidth=2)
ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title('Error vs λ', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_analysis.png', dpi=150)
plt.show()

print(f'\n📈 Figure saved')